In [ ]:
#| hide
from matching_separable_simuls.core import *

# matching_separable_simuls

> Simulations for Galichon-Salanie paper on estimating separable matching models

## Install

```sh
pip install matching_separable_simuls
```

## How to use

The package reads the marriage patterns from the Choo and Siow 2006 *Journal of Political Economy* paper for the non-reform states of the  1970 wave. It fits a parsimonious homoskedastic logit model. Then it generates $S$ datasets from the fitted model, and it uses them to test the estimation methods presented in Galichon-Salanié 2022.

In [ ]:
1+1

2